In [1]:
from marubatsu import NpIntBoard
from ai import ai_match, ai2

ai_match(ai=[ai2, ai2], mbparams={"boardclass": NpIntBoard}, match_num=100)

ai2 VS ai2


100%|██████████| 100/100 [00:00<00:00, 4662.41it/s]

count     win    lose    draw
    1      55      33      12
    2      31      57      12
total      86      90      24

ratio     win    lose    draw
    1   55.0%   33.0%   12.0%
    2   31.0%   57.0%   12.0%
total   43.0%   45.0%   12.0%



[('count',
  [{'win': 55, 'lose': 33, 'draw': 12},
   {'win': 31, 'lose': 57, 'draw': 12},
   {'win': 86, 'lose': 90, 'draw': 24}],
  '7d'),
 ('ratio',
  [{'win': 0.55, 'lose': 0.33, 'draw': 0.12},
   {'win': 0.31, 'lose': 0.57, 'draw': 0.12},
   {'win': 0.43, 'lose': 0.45, 'draw': 0.12}],
  '7.1%')]

In [2]:
from marubatsu import Marubatsu
from collections import defaultdict
from tqdm import tqdm

def ai_match(ai, params=[{}, {}], match_num=10000, mbparams={}):
    print(f"{ai[0].__name__} VS {ai[1].__name__}")
    
    mb = Marubatsu(**mbparams)

    # ai[0] VS ai[1] と ai[1] VS a[0] の対戦を match_num 回行い、通算成績を数える
    count_list = [ defaultdict(int), defaultdict(int)]
    for _ in tqdm(range(match_num)):
        count_list[0][mb.play(ai, params=params, verbose=False)] += 1
        count_list[1][mb.play(ai=ai[::-1], params=params[::-1], verbose=False)] += 1

    # ai[0] から見た通算成績を計算する
    count_list_ai0 = [
        # ai[0] VS ai[1] の場合の、ai[0] から見た通算成績
        { 
            "win": count_list[0][mb.CIRCLE],
            "lose": count_list[0][mb.CROSS],
            "draw": count_list[0][mb.DRAW],
        },
        # ai[1] VS ai[0] の場合の、ai[0] から見た通算成績
        { 
            "win": count_list[1][mb.CROSS],
            "lose": count_list[1][mb.CIRCLE],
            "draw": count_list[1][mb.DRAW],
        },
    ]           

    # 両方の対戦の通算成績の合計を計算する
    count_list_ai0.append({})
    for key in count_list_ai0[0]:
        count_list_ai0[2][key] = count_list_ai0[0][key] + count_list_ai0[1][key]

    # それぞれの比率を計算し、ratio_list に代入する
    ratio_list = [ {}, {}, {} ]
    for i in range(3):
        for key in count_list_ai0[i]:
            ratio_list[i][key] = count_list_ai0[i][key] / sum(count_list_ai0[i].values())
            
    # 各行の先頭に表示する文字列のリスト
    item_text_list = [ mb.board.MARK_TABLE[mb.CIRCLE], mb.board.MARK_TABLE[mb.CROSS], "total" ]    
    
    # 通算成績の回数と比率の表示
    width = max(len(str(match_num * 2)), 7)
    diff_list = [ ("count", count_list_ai0, f"{width}d"),
                  ("ratio", ratio_list, f"{width}.1%") ]
    for title, data, format in diff_list:
        print(title, end="")
        for key in data[0]:
            print(f" {key:>{width}}", end="")
        print()
        for i in range(3):
            print(f"{item_text_list[i]:5}", end="")
            for value in data[i].values():
                print(f" {value:{format}}", end="")
            print()
        print()
        
    return diff_list     

In [3]:
ai_match(ai=[ai2, ai2], mbparams={"boardclass": NpIntBoard}, match_num=100)

ai2 VS ai2


100%|██████████| 100/100 [00:00<00:00, 3923.72it/s]

count     win    lose    draw
o          60      28      12
x          28      59      13
total      88      87      25

ratio     win    lose    draw
o       60.0%   28.0%   12.0%
x       28.0%   59.0%   13.0%
total   44.0%   43.5%   12.5%



[('count',
  [{'win': 60, 'lose': 28, 'draw': 12},
   {'win': 28, 'lose': 59, 'draw': 13},
   {'win': 88, 'lose': 87, 'draw': 25}],
  '7d'),
 ('ratio',
  [{'win': 0.6, 'lose': 0.28, 'draw': 0.12},
   {'win': 0.28, 'lose': 0.59, 'draw': 0.13},
   {'win': 0.44, 'lose': 0.435, 'draw': 0.125}],
  '7.1%')]

In [4]:
mb = Marubatsu(boardclass=NpIntBoard)
t = tuple(mb.board.board)
print(t)
print({t: 0})

(array([0, 0, 0]), array([0, 0, 0]), array([0, 0, 0]))


TypeError: unhashable type: 'numpy.ndarray'

In [5]:
t = tuple(mb.board.board.flatten())
print(t)
print({t: 0})

(np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0))
{(np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0)): 0}


In [6]:
%timeit mb.board_to_str()
%timeit tuple(mb.board.board.flatten())
%timeit tuple(mb.board.board.flatten().tolist())

1.52 μs ± 27.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
1.62 μs ± 28.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
743 ns ± 18.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [7]:
b = bytes([1, 2, 3])
print(b)
print(b[0])

b'\x01\x02\x03'
1


In [8]:
bytes([300])

ValueError: bytes must be in range(0, 256)

In [9]:
b[0] = 5

TypeError: 'bytes' object does not support item assignment

In [10]:
print({b: 0})

{b'\x01\x02\x03': 0}


In [11]:
b = mb.board.board.tobytes()
print(b)
print({b: 0})

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
{b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00': 0}


In [12]:
%timeit mb.board.board.tobytes()

90.1 ns ± 3.6 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [13]:
from abc import abstractmethod
from marubatsu import Board, ListBoard, NpBoard

@abstractmethod
def board_to_hashable(self):
    pass

Board.board_to_hashable = board_to_hashable

def board_to_hashable(self):
    return self.board_to_str()

ListBoard.board_to_hashable = board_to_hashable

def board_to_hashable(self):
    return self.board.tobytes()

NpBoard.board_to_hashable = board_to_hashable

In [14]:
def board_to_hashable(self):
    return self.board.board_to_hashable()

Marubatsu.board_to_hashable = board_to_hashable

In [15]:
import numpy as np

def calc_same_hashables(self, move=None):
    data = [ [ 0,  0,  1, 1, -1,  0,  1,  0, -1, 0,  1,  0],
            [ 1, -1,  0, 1,  0, -1] * 2,
            [ 1,  0, -1, 0,  1,  0,  0,  0,  1, 1, -1,  0],
            [ 1, -1,  0, 0,  0,  1] * 2,
            [ 0,  1,  0, 1,  0, -1] * 2,
            [ 1,  0, -1, 1, -1,  0] * 2,
            [ 0,  0,  1, 0,  1,  0] * 2, ]
    if move is None:
        hashables = set([self.board_to_hashable()])
    else:
        hashables = { self.board_to_hashable(): move }
    for xa, xb, xc, ya, yb, yc, xa2, xb2, xc2, ya2, yb2, yc2 in data:
        txt = ""
        for x in range(self.BOARD_SIZE):
            for y in range(self.BOARD_SIZE):
                txt += self.getmark(xa * (self.BOARD_SIZE - 1) + xb * x + xc * y, ya * (self.BOARD_SIZE - 1) + yb * x + yc * y)
        if move is None:
            hashables.add(txt)
        else:
            x, y = self.move_to_xy(move)
            x, y = xa2 * (self.BOARD_SIZE - 1) + xb2 * x + xc2 * y, ya2 * (self.BOARD_SIZE - 1) + yb2 * x + yc2 * y
            hashables[txt] = self.xy_to_move(x, y)
    return hashables     

ListBoard.calc_same_hashables = calc_same_hashables

In [16]:
def calc_same_hashables(self, move=None):
    if move is None:
        hashables = set([self.board_to_hashable()])
    else:
        hashables = { self.board_to_hashable(): move }
    boardorig = self.board
    if move is not None:
        x, y = self.move_to_xy(move)
    for i in range(7):
        if i != 3:
            self.board = np.rot90(self.board)
        else:
            self.board = np.fliplr(self.board)
        hashable = self.board_to_hashable()
        if move is None:
            hashables.add(hashable)
        else:
            if i == 3:
                y = self.BOARD_SIZE - y - 1
            else:
                x, y = self.BOARD_SIZE - y - 1, x
            hashables[hashable] = self.xy_to_move(x, y)
    self.board = boardorig
    return hashables

NpBoard.calc_same_hashables = calc_same_hashables

In [ ]:
from marubatsu import List1dBoard, ArrayBoard
from ai import ai14s, ai_abs_dls

for boardclass in [ListBoard, List1dBoard, ArrayBoard, NpBoard, NpIntBoard]:
    for count_linemark in [False, True]:
        print(f"boardclass: {boardclass.__name__}, count_linemark {count_linemark}")
        mb = Marubatsu(boardclass=boardclass, count_linemark=count_linemark)
        eval_params = {"minimax": True}
        %timeit ai_abs_dls(mb, eval_func=ai14s, eval_params=eval_params, use_tt=True, maxdepth=8)

boardclass: ListBoard, count_linemark False
18 ms ± 255 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
boardclass: ListBoard, count_linemark True
17 ms ± 313 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
boardclass: List1dBoard, count_linemark False
18.2 ms ± 1.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
boardclass: List1dBoard, count_linemark True
16.7 ms ± 300 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
boardclass: ArrayBoard, count_linemark False
18.8 ms ± 390 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
boardclass: ArrayBoard, count_linemark True
18.1 ms ± 233 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
boardclass: NpBoard, count_linemark False
45.7 ms ± 2.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
boardclass: NpBoard, count_linemark True
40.3 ms ± 2.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
boardclass: NpIntBoard, count_linemark False
43.8 ms ± 2.21 ms per loop (mean ± std. de

In [18]:
mb = Marubatsu(boardclass=ListBoard)
l = []
for col in mb.board.board:
    l += col
t = tuple(l)
print(t)

('.', '.', '.', '.', '.', '.', '.', '.', '.')


In [19]:
%timeit mb.board_to_str()

478 ns ± 2.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [21]:
%%timeit

l = []
for col in mb.board.board:
    l += col
t = tuple(l)

270 ns ± 15.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [22]:
def board_to_hashable(self):
    l = []
    for col in self.board:
        l += col
    return tuple(l)

ListBoard.board_to_hashable = board_to_hashable

boardclass = ListBoard
for count_linemark in [False, True]:
    print(f"boardclass: {boardclass.__name__}, count_linemark {count_linemark}")
    mb = Marubatsu(boardclass=boardclass, count_linemark=count_linemark)
    eval_params = {"minimax": True}
    %timeit ai_abs_dls(mb, eval_func=ai14s, eval_params=eval_params, use_tt=True, maxdepth=8)

boardclass: ListBoard, count_linemark False
113 ms ± 3.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
boardclass: ListBoard, count_linemark True
105 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
t = {}
for i in range(100):
    t[i] = 0
    t[str(i)] = 0
    t[str(i) * 9] = 0
    t[str(i) * 100] = 0
    t[tuple([str(i)])] = 0
    t[tuple([str(i)] * 9)] = 0
    t[tuple([str(i)] * 100)] = 0
    t[bytes([i])] = 0
    t[bytes([i] * 9)] = 0
    t[bytes([i] * 100)] = 0

keylist = [0, "0", "0" * 9, "0" * 100, 
           tuple(["0"]), tuple(["0"] * 9), tuple(["0"] * 100),
           bytes([0]), bytes([0] * 9), bytes([0] * 100)]

for key in keylist:
    print(f"key = {key}")
    %timeit t[key]

key = 0
33.8 ns ± 1.09 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
key = 0
44.3 ns ± 1.26 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
key = 000000000
46.8 ns ± 1.75 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
key = 0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
48.9 ns ± 1.91 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
key = ('0',)
63.7 ns ± 2.08 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
key = ('0', '0', '0', '0', '0', '0', '0', '0', '0')
186 ns ± 12.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
key = ('0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0